# Marc's Data Playground!

This is my sandbox to work with the datasets and try out different visualization schemes!
Feel free to leave comments!

In [1]:
#Take care of the requisite imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas
import geoplot

# Election Data Loading/Cleaning/Merging Scripts
More or less what it says on the tin.

In [2]:
# Load the Harvard Precinct data from CSV and print out some useful classification info

harvard_data = pd.read_csv('../../Data/electoral_data/harvard_dataset/2016-precinct-house.csv', encoding='latin-1')
harvard_data.info()
harvard_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1127515 entries, 0 to 1127514
Data columns (total 37 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   year                   1127515 non-null  int64  
 1   stage                  1127515 non-null  object 
 2   special                1127515 non-null  bool   
 3   state                  1127515 non-null  object 
 4   state_postal           1127515 non-null  object 
 5   state_fips             1127515 non-null  int64  
 6   state_icpsr            1127515 non-null  int64  
 7   county_name            1123657 non-null  object 
 8   county_fips            1124269 non-null  float64
 9   county_ansi            1123657 non-null  float64
 10  county_lat             1123657 non-null  float64
 11  county_long            1123657 non-null  float64
 12  jurisdiction           1127515 non-null  object 
 13  precinct               1127515 non-null  object 
 14  candidate         

,year,stage,special,state,state_postal,state_fips,state_icpsr,county_name,county_fips,county_ansi,...,candidate_middle,candidate_full,candidate_suffix,candidate_nickname,candidate_fec,candidate_fec_name,candidate_google,candidate_govtrack,candidate_icpsr,candidate_maplight
0,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,H6AL02167,"MATHIS, NATHAN",NaN,NaN,NaN,NaN
4,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,Martha Roby,NaN,NaN,H0AL02087,NaN,kg:/m/0drx5mb,412394.0,21192.0,1408.0


In [3]:
# Perform some intial data slicing to extract pertinent info for NC

nc_precinct_county_party_votes = harvard_data[['state_postal', 'county_name', 'jurisdiction','precinct', 'mode', 'office', 'candidate_normalized', 'party', 'votes']].copy()
nc_precinct_county_party_votes = nc_precinct_county_party_votes[nc_precinct_county_party_votes['state_postal'].str.match('NC')]
nc_precinct_county_party_votes['loc_prec'] = nc_precinct_county_party_votes['jurisdiction'] + nc_precinct_county_party_votes['precinct']
nc_precinct_county_party_votes.info()
nc_precinct_county_party_votes.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26859 entries, 549492 to 576350
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   state_postal          26859 non-null  object
 1   county_name           26859 non-null  object
 2   jurisdiction          26859 non-null  object
 3   precinct              26859 non-null  object
 4   mode                  26859 non-null  object
 5   office                26859 non-null  object
 6   candidate_normalized  26859 non-null  object
 7   party                 26859 non-null  object
 8   votes                 26859 non-null  int64 
 9   loc_prec              26859 non-null  object
dtypes: int64(1), object(9)
memory usage: 2.3+ MB


,state_postal,county_name,jurisdiction,precinct,mode,office,candidate_normalized,party,votes,loc_prec
549492,NC,Alamance County,ALAMANCE,11,election day,US House,glidewell,democratic,387,ALAMANCE11
549493,NC,Alamance County,ALAMANCE,11,absentee by mail,US House,glidewell,democratic,18,ALAMANCE11
549494,NC,Alamance County,ALAMANCE,11,provisional,US House,glidewell,democratic,8,ALAMANCE11
549495,NC,Alamance County,ALAMANCE,11,one stop,US House,glidewell,democratic,714,ALAMANCE11
549496,NC,Alamance County,ALAMANCE,11,election day,US House,walker,republican,432,ALAMANCE11
549497,NC,Alamance County,ALAMANCE,11,absentee by mail,US House,walker,republican,36,ALAMANCE11
549498,NC,Alamance County,ALAMANCE,11,provisional,US House,walker,republican,2,ALAMANCE11
549499,NC,Alamance County,ALAMANCE,11,one stop,US House,walker,republican,581,ALAMANCE11
549500,NC,Alamance County,ALAMANCE,13,election day,US House,glidewell,democratic,371,ALAMANCE13
549501,NC,Alamance County,ALAMANCE,13,absentee by mail,US House,glidewell,democratic,20,ALAMANCE13


In [4]:
# Prepare a version of the dataset that's ready to be paired with the shapefile
# First, get only what we want and calculate victory margins

consolidated_prec = nc_precinct_county_party_votes[['loc_prec', 'party', 'votes']].copy()
consolidated_prec = consolidated_prec.groupby(['loc_prec', 'party']).sum()
consolidated_prec.to_csv('../../Data/electoral_data/wrangled_data/NC_consolidated_precinct.csv')
consolidated_prec = pd.read_csv('../../Data/electoral_data/wrangled_data/NC_consolidated_precinct.csv') #TODO: FIX THIS!!!!
consolidated_prec.head()

,loc_prec,party,votes
0,ALAMANCE01,democratic,448
1,ALAMANCE01,republican,1859
2,ALAMANCE02,democratic,387
3,ALAMANCE02,republican,2026
4,ALAMANCE035,democratic,920


# Precinct Data Loading/Cleaning/Merging Scripts
See above! :)

In [5]:
nc_precinct_geoframe = geopandas.read_file('../../Data/map_data/nc_2016')
nc_precinct_geoframe['loc_prec'] = nc_precinct_geoframe['COUNTY_NAM'] + nc_precinct_geoframe['PREC_ID']
nc_precinct_geoframe = nc_precinct_geoframe[['loc_prec', 'geometry']]
nc_precinct_HOR = consolidated_prec.merge(right=nc_precinct_geoframe, left_on='loc_prec', right_on='loc_prec', how='inner')
reshaped_nc_precinct_HOR = pd.DataFrame(data={'loc_prec':nc_precinct_HOR['loc_prec'].unique(), 'geometry':nc_precinct_HOR['geometry'].unique()})

# Get dem_votes
conditions = [
    (nc_precinct_HOR['party'] == 'democratic'),
    (nc_precinct_HOR['party'] != 'democratic')]
choices = [nc_precinct_HOR.get('loc_prec', 'votes'), 0]
print(np.select(conditions, choices, default=0))
# reshaped_nc_precinct_HOR['dem_votes'] = np.select(conditions, choices, default=0)


['ALAMANCE01' 0 'ALAMANCE02' ... 0 'YANCEY11 PRI' 0]


# Visualizations!

